<a href="https://colab.research.google.com/github/SnehaPandey-1720/Ds-Da-learning-hub/blob/main/Basic_Stats_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem Statement : Hospital Patient Data Analysis

● Context:
A hospital maintains patient records including admission details, department, diagnosis, doctor, and bill amount. You have two datasets: one with patient info and another with billing details. Some patients have blank bill amounts, and there are multiple rows for the same patient due to follow-ups.

● Tasks:
1.	Load the patient dataset and show summary with info().
2.	Select only the columns relevant for billing: ['PatientID', 'Department', 'Doctor', 'BillAmount'].
3.	Drop administrative columns like ['ReceptionistID', 'CheckInTime'].
4.	Use groupby to find total bill amount per department.
5.	Remove duplicate patient records based on PatientID.
6.	Fill missing BillAmount values with the mean bill amount.
7.	Merge the billing dataset with patient dataset on PatientID.
8.	Concatenate an additional DataFrame that contains new patients for the current week (row-wise).
9.	Concatenate new billing category columns like ['InsuranceCovered', 'FinalAmount'] (column-wise).


In [ ]:
# 1. Load datasets
import pandas as pd

In [ ]:
patients = pd.read_csv("Patient_Data.csv")      # patient dataset
patients.info()
patients

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PatientID       6 non-null      int64  
 1   Name            6 non-null      object 
 2   Department      6 non-null      object 
 3   Doctor          6 non-null      object 
 4   BillAmount      4 non-null      float64
 5   ReceptionistID  6 non-null      int64  
 6   CheckInTime     6 non-null      object 
dtypes: float64(1), int64(2), object(4)
memory usage: 468.0+ bytes


,PatientID,Name,Department,Doctor,BillAmount,ReceptionistID,CheckInTime
0,101,Alice,Cardiology,Dr. Smith,5000.0,1,2023-01-10 09:00
1,102,Bob,Neurology,Dr. John,NaN,2,2023-01-11 10:30
2,103,Charlie,Orthopedics,Dr. Lee,7500.0,1,2023-01-12 11:00
3,104,David,Cardiology,Dr. Smith,6200.0,3,2023-01-13 12:00
4,105,Eva,Dermatology,Dr. Rose,NaN,2,2023-01-14 08:45
5,101,Alice,Cardiology,Dr. Smith,5000.0,1,2023-01-10 09:00


In [ ]:
billing = pd.read_csv("Billing_Data.csv")        # billing dataset
billing.info()
billing

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   PatientID         5 non-null      int64
 1   InsuranceCovered  5 non-null      int64
 2   FinalAmount       5 non-null      int64
dtypes: int64(3)
memory usage: 252.0 bytes


,PatientID,InsuranceCovered,FinalAmount
0,101,2000,3000
1,102,1500,3500
2,103,2500,5000
3,104,3000,3200
4,105,1000,4000


In [ ]:
# 2. Select relevant billing columns
billing_relevant = patients[['PatientID', 'Department', 'Doctor', 'BillAmount']]
billing_relevant

,PatientID,Department,Doctor,BillAmount
0,101,Cardiology,Dr. Smith,5000.0
1,102,Neurology,Dr. John,NaN
2,103,Orthopedics,Dr. Lee,7500.0
3,104,Cardiology,Dr. Smith,6200.0
4,105,Dermatology,Dr. Rose,NaN
5,101,Cardiology,Dr. Smith,5000.0


In [ ]:
# 3. Drop unnecessary administrative columns
patients_cleaned = patients.drop(columns=['ReceptionistID', 'CheckInTime'], errors='ignore')
patients_cleaned

,PatientID,Name,Department,Doctor,BillAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0
1,102,Bob,Neurology,Dr. John,NaN
2,103,Charlie,Orthopedics,Dr. Lee,7500.0
3,104,David,Cardiology,Dr. Smith,6200.0
4,105,Eva,Dermatology,Dr. Rose,NaN
5,101,Alice,Cardiology,Dr. Smith,5000.0


In [ ]:
# 4. Total bill amount per department
dept_bill = billing_relevant.groupby('Department')['BillAmount'].sum().reset_index()
print("\nTotal Bill per Department:")
print(dept_bill)


Total Bill per Department:
    Department  BillAmount
0   Cardiology     16200.0
1  Dermatology         0.0
2    Neurology         0.0
3  Orthopedics      7500.0


In [ ]:
# 5. Remove duplicate patients (keep first occurrence)
patients_unique = patients_cleaned.drop_duplicates(subset='PatientID', keep='first')
patients_unique

,PatientID,Name,Department,Doctor,BillAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0
1,102,Bob,Neurology,Dr. John,NaN
2,103,Charlie,Orthopedics,Dr. Lee,7500.0
3,104,David,Cardiology,Dr. Smith,6200.0
4,105,Eva,Dermatology,Dr. Rose,NaN


In [ ]:
# 6. Fill missing BillAmount with mean
mean_bill = patients_unique['BillAmount'].mean()
patients_unique['BillAmount'] = patients_unique['BillAmount'].fillna(mean_bill)

C:\Users\Kaustubh Jadhav\AppData\Local\Temp\ipykernel_24540\845673225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patients_unique['BillAmount'] = patients_unique['BillAmount'].fillna(mean_bill)


In [ ]:
# 7. Merge patient and billing datasets on PatientID
merged = pd.merge(patients_unique, billing, on='PatientID', how='left')
merged.info()
merged

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   PatientID         5 non-null      int64  
 1   Name              5 non-null      object 
 2   Department        5 non-null      object 
 3   Doctor            5 non-null      object 
 4   BillAmount        5 non-null      float64
 5   InsuranceCovered  5 non-null      int64  
 6   FinalAmount       5 non-null      int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 412.0+ bytes


,PatientID,Name,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount
0,101,Alice,Cardiology,Dr. Smith,5000.000000,2000,3000
1,102,Bob,Neurology,Dr. John,6233.333333,1500,3500
2,103,Charlie,Orthopedics,Dr. Lee,7500.000000,2500,5000
3,104,David,Cardiology,Dr. Smith,6200.000000,3000,3200
4,105,Eva,Dermatology,Dr. Rose,6233.333333,1000,4000


In [ ]:
# 8. Concatenate new patients (row-wise)
new_patients = pd.DataFrame({
    'PatientID': [106, 107],
    'Name': ['Ravi Kumar', 'Sneha Joshi'],
    'Department': ['Cardiology', 'Orthopedics'],
    'Doctor': ['Dr. Smith', 'Dr. Lee'],
    'BillAmount': [5000, 7500],
    'InsuranceCovered': [2000,2500],
    'FinalAmount': [3000,2000] ,
})
merged = pd.concat([merged, new_patients], ignore_index=True)
merged

,PatientID,Name,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount
0,101,Alice,Cardiology,Dr. Smith,5000.000000,2000,3000
1,102,Bob,Neurology,Dr. John,6233.333333,1500,3500
2,103,Charlie,Orthopedics,Dr. Lee,7500.000000,2500,5000
3,104,David,Cardiology,Dr. Smith,6200.000000,3000,3200
4,105,Eva,Dermatology,Dr. Rose,6233.333333,1000,4000
5,106,Ravi Kumar,Cardiology,Dr. Smith,5000.000000,2000,3000
6,107,Sneha Joshi,Orthopedics,Dr. Lee,7500.000000,2500,2000


In [ ]:
# 9. Concatenate new billing columns (col-wise)
import numpy as np
extra_cols = pd.DataFrame({
    'GST': merged['FinalAmount'] * 0.18, # assume 20% covered by insurance
    'PaymentStatus': np.random.choice(['Paid', 'Unpaid'], size=len(merged))  # example data
})

final_dataset = pd.concat([merged, extra_cols], axis=1)

In [ ]:
# Final Output
# ---------------------------
print("\nFinal Cleaned Dataset:")
final_dataset


Final Cleaned Dataset:


,PatientID,Name,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount,GST,PaymentStatus
0,101,Alice,Cardiology,Dr. Smith,5000.000000,2000,3000,540.0,Unpaid
1,102,Bob,Neurology,Dr. John,6233.333333,1500,3500,630.0,Unpaid
2,103,Charlie,Orthopedics,Dr. Lee,7500.000000,2500,5000,900.0,Paid
3,104,David,Cardiology,Dr. Smith,6200.000000,3000,3200,576.0,Paid
4,105,Eva,Dermatology,Dr. Rose,6233.333333,1000,4000,720.0,Unpaid
5,106,Ravi Kumar,Cardiology,Dr. Smith,5000.000000,2000,3000,540.0,Paid
6,107,Sneha Joshi,Orthopedics,Dr. Lee,7500.000000,2500,2000,360.0,Paid
